In [2]:
import numpy as np
import torch
from tqdm import tqdm
import os


In [24]:
'''
choose between train and val
'''
# filename = 'val'
filename = 'train'

In [34]:
def f(a,b):
    assert len(a) == len(b)
    return [a[b[i]-1] for i in range(len(b))]

def permute(list_of_permutations,partial_sum=True):
    answer = list(range(1,len(list_of_permutations[0])+1))
    for p in list_of_permutations:
        if partial_sum:
            answer += f(answer[-len(list(p)):], list(p))
        else:
            answer = f(answer, list(p))
    return answer

In [37]:
rng = np.random.default_rng(1234)
block_size = 64
p = 5
C = block_size // (2 * (p+2))
N = 10
random_list = rng.permuted(np.tile(np.arange(1,p+1), N * C).reshape(N*C,p), axis=1).reshape(N,C,p)
input_length_list = np.random.randint(low = 1,high =C+1, size = (N,), dtype = np.int8)
token_id_list = []
for i in range(N):
    current_id = []
    current_input_tokens = random_list[i]
    for j in range(C):
        current_id += [-3]+ list(current_input_tokens[j])+ [-4]
    token_id_list.append(current_id + [-2] + list(permute(random_list[i],partial_sum=True)))
print( token_id_list)

[[-3, 1, 5, 2, 4, 3, -4, -3, 2, 4, 3, 1, 5, -4, -3, 1, 2, 4, 3, 5, -4, -3, 2, 1, 5, 4, 3, -4, -2, 1, 2, 3, 4, 5, 1, 5, 2, 4, 3, 5, 4, 2, 1, 3, 5, 4, 1, 2, 3, 4, 5, 3, 2, 1], [-3, 2, 1, 3, 4, 5, -4, -3, 5, 1, 4, 2, 3, -4, -3, 1, 3, 2, 4, 5, -4, -3, 2, 4, 3, 5, 1, -4, -2, 1, 2, 3, 4, 5, 2, 1, 3, 4, 5, 5, 2, 4, 1, 3, 5, 4, 2, 1, 3, 4, 1, 2, 3, 5], [-3, 3, 5, 2, 4, 1, -4, -3, 5, 3, 2, 4, 1, -4, -3, 4, 2, 3, 1, 5, -4, -3, 5, 3, 4, 1, 2, -4, -2, 1, 2, 3, 4, 5, 3, 5, 2, 4, 1, 1, 2, 5, 4, 3, 4, 2, 5, 1, 3, 3, 5, 1, 4, 2], [-3, 3, 2, 4, 5, 1, -4, -3, 3, 2, 1, 5, 4, -4, -3, 1, 4, 3, 5, 2, -4, -3, 3, 2, 5, 1, 4, -4, -2, 1, 2, 3, 4, 5, 3, 2, 4, 5, 1, 4, 2, 3, 1, 5, 4, 1, 3, 5, 2, 3, 1, 2, 4, 5], [-3, 2, 3, 5, 1, 4, -4, -3, 3, 1, 5, 2, 4, -4, -3, 4, 3, 5, 2, 1, -4, -3, 1, 2, 5, 3, 4, -4, -2, 1, 2, 3, 4, 5, 2, 3, 5, 1, 4, 5, 2, 4, 3, 1, 3, 4, 1, 2, 5, 3, 4, 5, 1, 2], [-3, 2, 5, 4, 3, 1, -4, -3, 4, 5, 2, 1, 3, -4, -3, 1, 3, 5, 2, 4, -4, -3, 4, 1, 3, 2, 5, -4, -2, 1, 2, 3, 4, 5, 2, 5, 4, 3, 1, 3, 1, 5

In [44]:
token_id_list[0][:-1]

[5, -2]

In [26]:
dtype = np.int8
arr = np.memmap(filename + '.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
arr_label = np.memmap(filename + '_label.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
print(f"writing {filename}...")
for jj, example in tqdm(enumerate(token_id_list)):
    arr[jj * block_size : jj * block_size + len(example)-(p+2)] = example[:-(p+2)]
    arr[jj * block_size + len(example)-1 : (jj + 1) * block_size] = 0
    arr_label[jj * block_size : (1+jj) * block_size ] = -1
    arr_label[jj * block_size + (p+2)*C:jj * block_size + (p+2)*(C+1)] = example[-(p+2):]
    # pad to bock_size
arr.flush()
arr_label.flush()

writing train...


1000000it [00:12, 81239.66it/s]


In [21]:
print(arr[:200])

[-3  1  5  2  4  3 -4 -3  2  4  3  1  5 -4 -3  1  2  4  3  5 -4 -3  2  1
  5  4  3 -4 -3  2  1  3  4  5 -4 -3  5  1  4  2  3 -4 -3  1  3  2  4  5
 -4 -3  2  4  3  5  1 -4 -3  3  5  2  4  1 -4 -3  5  3  2  4  1 -4 -3  4
  2  3  1  5 -4 -3  5  3  4  1  2 -4 -3  3  2  4  5  1 -4 -3  3  2  1  5
  4 -4 -3  1  4  3  5  2 -4 -3  3  2  5  1  4 -4 -3  2  3  5  1  4 -4 -3
  3  1  5  2  4 -4 -2  4  5  3  1  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [5]:
data_dir = os.path.join('data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
train_label = np.memmap(os.path.join(data_dir, 'train_label.bin'), dtype=np.int16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
val_label = np.memmap(os.path.join(data_dir, 'val_label.bin'), dtype=np.int16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    label = train_label if split == 'train' else val_label
    ix = torch.randint(len(data) // block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((label[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

In [5]:
random_ints = np.random.randint(low = 0,high =2, size = 100, dtype = np.int8)
exp = ''
for i in random_ints[:-1]:
    exp += str(i) + "+"
exp += random_ints[:-1] + '='
print(exp)
random_ints.sum()

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int8'), dtype('<U1')) -> None

In [7]:
random_ints

array([0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1], dtype=int8)

In [4]:
print(1+1+1+0+1+1+1+0+1+1+0+0+1+0+1+1+0+1+0+0+1+1+0+0+1+0+1+0+1+1+0+0+1+0+0+0+1+0+0+0+1+0+1+0+0+1+0+0+0+1+0+1+1+0+0+1+1+0+1+1+1+0+0+0+1+1+1+0+0+1+1+1+0+0+0+1+1+0+0+0+1+0+1+0+0+1+1+0+1+1+1+1+0+1+0+1+1+0+0+0)

50


In [6]:
np.random.randn(100)

array([ 0.10775194,  0.82486141,  1.39492264, -0.27424914,  0.62799271,
       -0.67446664, -0.96435732,  1.24493219,  0.09221922,  0.43513877,
       -1.91110539,  0.79870031,  0.38872974, -2.07428724, -0.60261176,
       -1.07780889, -0.56687404, -0.60352224,  1.28743884,  1.56997499,
        0.23927013, -0.53352818, -1.00921134, -0.97632384,  0.39886744,
       -0.62070398, -0.76408363,  0.9848463 , -0.50201059,  0.33955341,
        0.63073409, -0.25691401,  0.49072394, -0.62182025, -0.12275391,
        0.83969668,  0.8973218 ,  3.09926477,  1.82240243, -1.01629954,
       -0.06565471,  0.73316766,  0.89292083,  1.51522196, -1.24694117,
        1.43019879, -1.64006693, -0.3832247 , -0.66819771, -1.55994517,
       -0.15607834,  0.8659179 , -0.07323536,  1.00829473, -0.02956538,
       -0.11243826, -0.34180768,  1.30008599, -0.97497401, -0.95254486,
        0.02505601, -0.98974753, -0.92236767,  0.2877739 , -0.35125652,
        1.12736306, -0.79398367, -1.05277052,  0.31783241, -0.59

In [8]:

a = np.array([0.10775194,  0.82486141,  1.39492264, -0.27424914,  0.62799271,
       -0.67446664, -0.96435732,  1.24493219,  0.09221922,  0.43513877,
       -1.91110539,  0.79870031,  0.38872974, -2.07428724, -0.60261176,
       -1.07780889, -0.56687404, -0.60352224,  1.28743884,  1.56997499,
        0.23927013, -0.53352818, -1.00921134, -0.97632384,  0.39886744,
       -0.62070398, -0.76408363,  0.9848463 , -0.50201059,  0.33955341,
        0.63073409, -0.25691401,  0.49072394, -0.62182025, -0.12275391,
        0.83969668,  0.8973218 ,  3.09926477,  1.82240243, -1.01629954,
       -0.06565471,  0.73316766,  0.89292083,  1.51522196, -1.24694117,
        1.43019879, -1.64006693, -0.3832247 , -0.66819771, -1.55994517,
       -0.15607834,  0.8659179 , -0.07323536,  1.00829473, -0.02956538,
       -0.11243826, -0.34180768,  1.30008599, -0.97497401, -0.95254486,
        0.02505601, -0.98974753, -0.92236767,  0.2877739 , -0.35125652,
        1.12736306, -0.79398367, -1.05277052,  0.31783241, -0.59624527,
        1.44136003, -0.90282434, -0.70868632, -1.98931671, -1.25867585,
       -0.95658665, -0.78425377,  0.39667268,  0.87046469,  0.10261665,
       -0.13300676,  0.5717015 , -0.76447064,  0.22568388,  0.61336665,
        0.01081189,  1.14641535,  0.22427346,  0.69834502,  0.85920204,
        0.52654736, -0.70958207, -1.03240437, -1.42778429,  0.33488912,
        0.09581042,  1.15791853, -0.88970196, -0.23684442,  0.67291775])

In [11]:
a

array([-2.07428724, -1.98931671, -1.91110539, -1.64006693, -1.55994517,
       -1.42778429, -1.25867585, -1.24694117, -1.07780889, -1.05277052,
       -1.03240437, -1.01629954, -1.00921134, -0.98974753, -0.97632384,
       -0.97497401, -0.96435732, -0.95658665, -0.95254486, -0.92236767,
       -0.90282434, -0.88970196, -0.79398367, -0.78425377, -0.76447064,
       -0.76408363, -0.70958207, -0.70868632, -0.67446664, -0.66819771,
       -0.62182025, -0.62070398, -0.60352224, -0.60261176, -0.59624527,
       -0.56687404, -0.53352818, -0.50201059, -0.3832247 , -0.35125652,
       -0.34180768, -0.27424914, -0.25691401, -0.23684442, -0.15607834,
       -0.13300676, -0.12275391, -0.11243826, -0.07323536, -0.06565471,
       -0.02956538,  0.01081189,  0.02505601,  0.09221922,  0.09581042,
        0.10261665,  0.10775194,  0.22427346,  0.22568388,  0.23927013,
        0.2877739 ,  0.31783241,  0.33488912,  0.33955341,  0.38872974,
        0.39667268,  0.39886744,  0.43513877,  0.49072394,  0.52